# Demo Visualization for reading level of Shrek characters

### Shane Caldwell

In [13]:
# Relevant Imports
from textstat.textstat import textstat
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("whitegrid")

%matplotlib inline

Goal here is to get the reading level of major characters. I will be plotting the The The Flesch-Kincaid Grade Level
of the character (found with flesch_kincaid_grade(dialogue)) by number of lines they have. 

Create one for each movie. See how it changes over time if it does. In d3 you should be able to select a film and have everything move in an attractive way organically, without harsh reloads. 

In [14]:
df = pd.read_csv("../data/shrek_dialog_w_timestamps.csv", index_col="Unnamed: 0")


In [39]:
#What characters do we have?
df.groupby('name').count().max(1).sort_values(ascending=False)

name
SHREK                    233
PRINCE CHARMING          135
ARTIE                    106
DONKEY                    95
MUSIC                     65
FIONA                     65
PUSS                      48
MERLIN                    48
SNOW WHITE                29
KING HAROLD               20
PINOCCHIO                 18
RAPUNZEL                  17
CAPTAIN HOOK              14
LANCELOT                  12
GINGERBREAD MAN           12
PIG                        8
DORIS                      8
QUEEN                      8
PRINCIPAL PYNCHLEY         6
CINDERELLA                 6
SLEEPING BEAUTY            6
GUINEVERRE                 5
CYCLOPS                    5
SHIP CAPTAIN               5
NANNY DWARF                4
RUMPLESTILTSKIN            4
EVIL TREE                  4
CHEERLEADERS               4
GUARD                      3
AUDIENCE                   3
                        ... 
MASTER OF CEREMONIES       2
MABEL                      2
WOLF                       2
CINDERALL

In [42]:
shrek_lines = df[df.name=='SHREK']
shrek_speech = ""
for index, row in shrek_lines.iterrows():
    shrek_speech += row['dialog'] + " "
print(textstat.flesch_kincaid_grade(shrek_speech))

fiona_lines = df[df.name=='FIONA']
fiona_speech = ""
for index, row in fiona_lines.iterrows():
    fiona_speech += row['dialog'] + " "
print(textstat.flesch_kincaid_grade(fiona_speech))

donkey_lines = (df[df.name=='DONKEY']).fillna(value="")
donkey_speech = ""
for index, row in donkey_lines.iterrows():
    donkey_speech += row['dialog'] + " "
print(textstat.flesch_kincaid_grade(donkey_speech))

puss_lines = (df[df.name=='PUSS']).fillna(value= "")
puss_speech = ""
for index, row in puss_lines.iterrows():
    puss_speech += row['dialog'] + " "
print(textstat.flesch_kincaid_grade(puss_speech))

charming_lines = df[df.name=='PRINCE CHARMING']
charming_lines = charming_lines.fillna(value = "")
charming_speech = ""
for index, row in charming_lines.iterrows():
    charming_speech += row['dialog'] + " "
print(textstat.flesch_kincaid_grade(charming_speech))

1.6
1.4
2.7
2.6
2.6


So let's write a function to get us those scores for every single character 


In [45]:
def reading_score_for_character(df, character_name):
    char_lines = df[df.name==character_name].fillna(value = "")
    char_speech = ""
    for index, row in char_lines.iterrows():
        char_speech += row['dialog'] + " "
    return textstat.flesch_kincaid_grade(char_speech)

1.6
2.7


In [55]:
names = df.name.unique()
reading_level = [["name","reading_level"]]
for name in names:
    score = reading_score_for_character(df, name)
    reading_level.append([name, score])

headers = reading_level.pop(0)
read_df = pd.DataFrame(reading_level, columns=headers)

read_df.sort_values(by = "reading_level", ascending=False)

,name,reading_level
62,OGRE BABIES,20.2
28,OGRE BABY,20.2
33,TIFFANY,17.0
17,MUSIC,9.9
12,FIDDLESWORTH,8.8
57,EVIL KNIGHT,7.4
24,PUPPET MASTER,6.8
3,ACTRESS,6.8
56,ANNOUNCER,6.6
6,WAITER,6.4
